# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [68]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [69]:
# checking the current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [70]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''): # skip empty row
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [71]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion

## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## We start running codes to create tables for below queries. Remember the principal: one table needs one query

#### Creating a Cluster

In [72]:
# create a cluster instance

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [73]:
try:
    session.execute("""
                CREATE KEYSPACE IF NOT EXISTS project2
                WITH REPLICATION =
                {'class':'SimpleStrategy' ,'replication_factor':1}
                """)
except Exception as e:
    print("Cannot create keyspace")
    print(e)

#### Set Keyspace

In [74]:
try:
    session.set_keyspace('project2')
except Exception as e:
    print('Cannot set keyspace')
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra, we think about queries before creating tables

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Question 1: We go through a process of 3 steps: create table, insert data, and query data

##### First, we create a table with Partition key is sessionId, and clustering key is iteminsession

In [75]:
session.execute("DROP TABLE IF EXISTS music_library")

query = """
                CREATE TABLE IF NOT EXISTS music_library(sessionId int, 
                                                         itemInSession int,
                                                         artist varchar,
                                                         song varchar,
                                                         length float,
                                                         PRIMARY KEY(sessionId, itemInSession)
                                                         );
        """
try:
    session.execute(query)       
except Exception as e:
    print(e)

##### Write the data to the table

In [76]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library (sessionId, itemInSession, artist, song, length) "
        query = query + "VALUES(%s, %s, %s, %s, %s);"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9],float(line[5])))

##### Do a SELECT to verify that the data have been inserted into each table

In [77]:
query = "SELECT artist, song, length FROM music_library WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    #rint(dir(row))
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### We repeat the same process for question 2 and 3

#### Question 2: Provide the name of artist, song title, and fullname of user for given userid and sessionid

##### We create a table with partition key is (userid, session id) and clustering key is iteminsession

In [78]:
session.execute("DROP TABLE IF EXISTS user_library")

query = """
            CREATE TABLE IF NOT EXISTS user_library(userid int,
                                                    sessionid int,
                                                    iteminsession int,
                                                    artist varchar,
                                                    song varchar,
                                                    first_name varchar,
                                                    last_name varchar,
                                                    PRIMARY KEY((userid, sessionid), iteminsession));
"""

try:
    session.execute(query)
except Exception as e:
    print(e)


##### Insert data to table

In [79]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_library (userid, sessionid, iteminsession, artist, song, first_name, last_name) "
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s);"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9],line[1], line[4]))
        

##### Do a SELECT to verify the table

In [80]:
query = "SELECT artist, song, first_name, last_name FROM user_library WHERE userid = 10 AND sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    #rint(dir(row))
    print(row.artist, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### Question 3: Provide fullname of users who listened a specific song.

##### Create a table with partition key is song, and clustering key is userid

In [81]:
session.execute("DROP TABLE IF EXISTS listen_history")

query = """
            CREATE TABLE IF NOT EXISTS listen_history(song varchar,
                                                      userid int,
                                                      first_name varchar,
                                                      last_name varchar,
                                                      PRIMARY KEY(song, userid));
"""

try:
    session.execute(query)
except Exception as e:
    print(e)              

##### Write data to table

In [82]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO listen_history (song, userid, first_name, last_name) "
        query = query + "VALUES(%s, %s, %s, %s);"
        session.execute(query, (line[9], int(line[10]), line[1],line[4]))

##### Do a SELECT to verify the inserted data

In [83]:
query = "SELECT first_name, last_name FROM listen_history WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    #rint(dir(row))
    print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [84]:
tables = ['music_library', 'user_library','listen_history']
for table in tables:
    query = "DROP TABLE IF EXISTS " + table
    session.execute(query)

### Close the session and cluster connection¶

In [85]:
session.shutdown()
cluster.shutdown()